#### The original squirrel banking setup assumes the decision to produce offspring yields an immediate payout: you get pregnant, and immediately have an offspring. We now extend to insist that suirrels are pregnant for a day before giving birth; if your food runs below zero while pregnant, both you and your offspring die.

In [1]:
import plotly.express as px
import pandas as pd

In [12]:
p0, p1, p2 = var('p0, p1, p2')

In [18]:
def sym_Lnow(p0, p1, p2, double_pregnancy = False):
    L = zero_matrix(SR, 2)
    L[0,0] = p1
    L[1, 0] = p2
    if double_pregnancy == False:
        L[0, 1] = 2*(p1+p2)
        L[1, 1] = 0
    else:
        L[0, 1] = 2*p1 + p2
        L[1, 1] = p2
    return L

def sym_Lpatient(p0, p1, p2, double_pregnancy = False):
    L = zero_matrix(SR, 3)
    L[0,0] = p1
    L[1,0] = p2
    L[2, 0] = 0
    L[0, 1] = p0
    L[1, 1] = p1
    L[2, 1] = p2 
    L[0, 2] = 1 + p0
    if double_pregnancy == False:
        L[1, 2] = p1 + p2
        L[2, 2] = 0
    else:
        L[1, 2] = p1
        L[2, 2] = p2
    return L

def sym_L_betahat(p0, p1, p2, betahat, double_pregnancy = False):
    if betahat == 0:
        L = sym_Lnow(p0, p1, p2, double_pregnancy)
    elif betahat == 1:
        L = sym_Lpatient(p0, p1, p2, double_pregnancy)
    else:
        L = zero_matrix(SR, betahat + 2)
        for i in range(betahat+1):
            L[i, i] = p1
            if i < betahat+1:
                L[i + 1, i] = p2
                if i > 0:
                    L[i-1, i] = p0
        L[0, betahat+1] = 1
        L[betahat - 1, betahat+1] = p0
        if double_pregnancy == False:
            L[betahat, betahat+1] = p1 + p2
            L[betahat+1, betahat+1] = 0
        else:
            L[betahat, betahat + 1] = p1
            L[betahat+1, betahat+1] = p2           
    return(L)

In [23]:
sym_L_betahat(p0, p1, p2, 3)

[     p1      p0       0       0       1]
[     p2      p1      p0       0       0]
[      0      p2      p1      p0      p0]
[      0       0      p2      p1 p1 + p2]
[      0       0       0      p2       0]

In [101]:
def normalize(P0, P1, P2):
    p0 = P0/(P0 + P1 + P2)
    p1 = P1/(P0 + P1 + P2)
    p2 = 1 - p0 - p1
    return([p0, p1, p2])

def Lnow(p0, p1, p2,s = 1, double_pregnancy = False):
    L = zero_matrix(QQ, 2)
    L[0,0] = p1*s
    L[1, 0] = p2*s
    if double_pregnancy == False:
        L[0, 1] = 2*(p1+p2)*s
        L[1, 1] = 0
    else:
        L[0, 1] = (2*p1 + p2)*s
        L[1, 1] = p2*s
    return L

def Lpatient(p0, p1, p2,s = 1, double_pregnancy = False):
    L = zero_matrix(QQ, 3)
    L[0,0] = p1*s
    L[1,0] = p2*s
    L[2, 0] = 0
    L[0, 1] = p0*s
    L[1, 1] = p1*s
    L[2, 1] = p2 *s
    L[0, 2] = (1 + p0)*s
    if double_pregnancy == False:
        L[1, 2] = (p1 + p2)*s
        L[2, 2] = 0
    else:
        L[1, 2] = p1*s
        L[2, 2] = p2*s
    return L

def L_betahat(p0, p1, p2, betahat, double_pregnancy = False, s = 1):
    normalized = normalize(p0, p1, p2)
    p0 = normalized[0]
    p1 = normalized[1]    
    p2 = normalized[2]
    if betahat == 0:
        L = Lnow(p0, p1, p2,s, double_pregnancy)
    elif betahat == 1:
        L = Lpatient(p0, p1, p2,s, double_pregnancy)
    else:
        L = zero_matrix(QQ, betahat + 2)
        for i in range(betahat+1):
            L[i, i] = p1
            if i < betahat+1:
                L[i + 1, i] = p2*s
                if i > 0:
                    L[i-1, i] = p0*s
        L[0, betahat+1] = 1*s
        L[betahat - 1, betahat+1] = p0*s
        if double_pregnancy == False:
            L[betahat, betahat+1] = (p1 + p2)*s
            L[betahat+1, betahat+1] = 0
        else:
            L[betahat, betahat + 1] = p1*s
            L[betahat+1, betahat+1] = p2*s         
    return(L)

def get_leading_eigenvalue(L):
    evals = L.eigenvalues()
    moduli = [float(e) for e in evals if e in RR]
    moduli = [e for e in moduli if e >= 0]
#    moduli = [abs(e) for e in evals]
    r = max(moduli)
    return(r)


def fitness(p0, p1, p2, betahat, double_pregnancy = False, s = 1):
    normalized = normalize(p0, p1, p2)
    p0 = normalized[0]
    p1 = normalized[1]
    p2 = normalized[2]
    L = L_betahat(p0, p1, p2, betahat, double_pregnancy, s)
    r = get_leading_eigenvalue(L)
    return(r)

def first_five_fitness(p0, p1, p2, double_pregnancy = False, s = 1):
    fitnesses = []
    for i in range(20):
        fitnesses.append(fitness(p0, p1, p2, i, double_pregnancy, s))
    return(fitnesses)
        

In [102]:
first_five_fitness(36, 26, 38.5,False)

[0.8424119698899946,
 0.9808245740441845,
 0.9935478975693632,
 0.9982153369812194,
 1.0002447479774834,
 1.001211502038509,
 1.0016900970615854,
 1.0019238220592825,
 1.0020271452474407,
 1.0020582533297175,
 1.0020484961721063,
 1.0020155283524623,
 1.0019696291476943,
 1.0019169304071789,
 1.001861148549049,
 1.001804552802041,
 1.0017485254174763,
 1.001693895334117,
 1.0016411418351567,
 1.0015905214405292]

In [103]:
first_five_fitness(31, 38, 31, double_pregnancy = False)

[0.871102048154313,
 0.9763953680972295,
 0.9877454676425148,
 0.9925330163760127,
 0.9949830556127044,
 0.9964003778575083,
 0.8314419583831054,
 0.9978903560054913,
 0.9983100622884657,
 0.998615998285155,
 0.9988458355122984,
 0.999022858555574,
 0.9991620856805092,
 0.9992735546710363,
 0.9993641804177775,
 0.9994388519189326,
 0.999501104774758,
 0.9995535470024477,
 0.99959813639554,
 0.9996363657527354]

In [88]:
for i in range(100):
    k = 1 - i/100
    f2 = first_five_fitness(34, 18, 48, double_pregnancy=False, s=k)[3]
    f3 = first_five_fitness(34, 18, 48, double_pregnancy=False, s= k)[4]
    if f3>f2:
        print(i)
        break

41


#### What we see is that an age independence survival rate s < 1 can alter betahat. Compare first_five_fitness(1, 1, 2, double_pregnancy = False, s) for $s\in\{1, 0.9\}$

#### Even more: we can get betaht = 3. Compute first_five_fitness(34, 18, 48, double_pregnancy = False, s = 0.6) and compare. Taking s = 0.59 in the above even get us betahat = 4. At  s = 0.5 we get betahat = 5.

In [193]:
n = 70
triples = []
for i in range(n + 1):
    first = n - i
    for j in range(n - first + 1):
        second = j
        third = i - j
        trip = [first, second, third]
        triples.append(trip)

first = []
second = []
third = []
for i in range(n + 1):
    for j in range(i + 1):
        first.append(n - i)
        second.append(j)
        third.append(i - j)

In [194]:
len(first)

2556

In [195]:
dbl_preg = False
fitness_2 = []
fitness_1 = []
fitness_0 = []
for k in range(len(first)):
    f0 = fitness(first[k], second[k], third[k], 0, dbl_preg)
    f2 = fitness(first[k], second[k], third[k], 2, dbl_preg)
    f1 = fitness(first[k], second[k], third[k], 1, dbl_preg)
    fitness_2.append(f2)
    fitness_1.append(f1)
    fitness_0.append(f0)
#    print(k)

optimal_betahat = []
for k in range(len(fitness_2)):
    f0, f1, f2 = fitness_0[k], fitness_1[k], fitness_2[k]
    if f0 <= f1:
        if f1<= f2:
            optimal_betahat.append(2)
        else:
            optimal_betahat.append(1)
    else:
        optimal_betahat.append(0)


    

        
    

In [196]:
unital_fitness = deepcopy(optimal_betahat)
precision = 0.01
for k in range(len(optimal_betahat)):
    if (1 - precision < fitness_0[k] < 1 + precision) or (1 - precision < fitness_1[k] < 1 + precision) or (1 - precision < fitness_2[k] < 1 + precision):
        unital_fitness[k] = 'unit fitness'
    else:
        pass

In [197]:
data_dict = {'no nuts':first, 'one nut':second, 'two nuts':third, 'Optimal betahat':optimal_betahat, 'fitness 1':fitness_1, 'fitness 2':fitness_2, 'fitness 0': fitness_0, 'unit fitness':unital_fitness}
my_df = pd.DataFrame.from_dict(data_dict)

In [198]:
def is_unit_fitness(row):
    if row['unit fitness 0'] == True:
        return(0)
    elif row['unit fitness 1'] == True:
        return(1)
    elif row['unit fitness 2'] == True:
        return(2)
    else:
        return(-1)
